# M3 Project

In this project, you will compare the performance of uninformed and informed search algorithms for the 4-sided dominoes problem. You will be given N<sup>2</sup> tiles (N = {2,3,4}) and will be asked to arrange them in an NxN grid in a way that adjacent tiles have the same number in their neighboring sides. The tiles cannot be rotated. See the example below for N=3.

<img src="m3project.png" width="400"/>

An initial version of the code with the problem specification (below) and a report template (at the bottom) are available in this notebook. Deliverables are the final code (non-functioning code is worth 0 points) and the comparison report.

Solve the task above using:
- one uninformed search algorithm of your choice (20pts)
- one informed search algorithm of your choice (20pts)

For your solution, describe the:
- search state space (10pts)
- successor function (10pts)
- heuristic function for the informed search (10pts)

Run each algorithm with at least 10 different initial states for each value of N, and compare the performance of the chosen algorithms for different puzzle sizes in terms of:
- average number of expanded states (15pts)
- success rate (15pts)

You are free to set a maximum number of expanded states for each algorithm, and return failure in case this number is reached.

# Implementation

You are free to change the code below as needed.

In [13]:
import random
from queue import Queue
import numpy as np 
import heapq

In [15]:
class Domino():
    """
    Implementation of a 4-sided domino tile.

    Methods
    -------
    is_above(other)
        Checks of the tile is above the other tile.
    is_under(other)
        Checks of the tile is under the other tile.
    is_on_the_left_of(other)
        Checks of the tile is on the left of the other tile.
    is_on_the_right_of(other)
        Checks of the tile is on the right of the other tile.
    """
    def __init__(self, top: int, right: int, bottom: int, left: int):
        assert isinstance(top, int) and isinstance(right, int) and isinstance(bottom, int) and isinstance(left, int), "Invalid tile value!"
        self.top = top
        self.right = right
        self.bottom = bottom
        self.left = left

    def is_above(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.bottom == other.top
    
    def is_under(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.top == other.bottom
    
    def is_on_the_left_of(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.right == other.left
    
    def is_on_the_right_of(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.left == other.right

In [17]:
import random

class FourDominoes:
    """
    Implementation of the 4-sided dominoes puzzle.

    Methods
    -------
    show()
        Visualize the current state.
    move(action)
        Apply an action to the current state.
    successor(state)
        Finds the list of successors for a given state.
    goal_test(state)
        Checks if the current state is a goal state.
    """

    def __init__(self, N, state=None):
        """
        Parameters
        ----------
        N
            Grid size (puzzle contains N^2 tiles)
        state
            Initial state configuration. If None is provided, a random one is created.
        """
        assert N >= 2 and N <= 4, "Invalid grid size!"
        self.N = N

        if state is not None:
            assert len(state) == self.N, "Invalid state size!"
            for row in state:
                assert len(row) == self.N, "Invalid state size!"
                for tile in row:
                    assert isinstance(tile, Domino), "Invalid state type!"
            self.state = state
        else:
            self.state = self.__get_random_state()
    
    def __get_random_state(self):
        """
        Generates a random puzzle configuration (grid of dominoes)
        """
        temp = []
        for i in range(self.N):
            for j in range(self.N):
                domino = Domino(
                    random.randint(1, 9) if i == 0 else temp[(i - 1) * self.N + j].bottom, # top
                    random.randint(1, 9),  # right
                    random.randint(1, 9),  # bottom
                    random.randint(1, 9) if j == 0 else temp[i * self.N + j - 1].right  # left
                )
                temp.append(domino)
        random.shuffle(temp)  # Shuffle to create a randomized configuration
        return tuple(tuple(temp[i * self.N:(i + 1) * self.N]) for i in range(self.N))

    def show(self):
        """ Prints the current state of the grid. """
        print('╔', end='')
        for i in range(self.N):
            print('═══', end='╦' if i < self.N - 1 else '╗\n')
        for i in range(self.N):
            print('║', end='')
            for j in range(self.N):
                print('╲{}╱║'.format(self.state[i][j].top), end='' if j < self.N - 1 else '\n')
            print('║', end='')
            for j in range(self.N):
                print('{}╳{}║'.format(self.state[i][j].left, self.state[i][j].right), end='' if j < self.N - 1 else '\n')
            print('║', end='')
            for j in range(self.N):
                print('╱{}╲║'.format(self.state[i][j].bottom), end='' if j < self.N - 1 else '\n')
            print('╠' if i < self.N - 1 else '╚', end='')
            for j in range(self.N):
                print('═══', end='╬' if i < self.N - 1 and j < self.N - 1 else '╩' if i == self.N - 1 and j < self.N - 1 else '╣\n' if i < self.N - 1 else '╝\n')

    def move(self, action):
        """
        Uses a given action to update the current state of the puzzle.
        """
        (r1, c1), (r2, c2) = action
        temp = [list(row) for row in self.state]
        temp[r1][c1], temp[r2][c2] = temp[r2][c2], temp[r1][c1]
        self.state = tuple(tuple(x) for x in temp)

    def successor(self, state):
        """
        Finds the list of successors for a given state.
        """
        successors = []
        for i in range(self.N * self.N):
            r1, c1 = divmod(i, self.N)
            for j in range(i + 1, self.N * self.N):
                r2, c2 = divmod(j, self.N)
                action = ((r1, c1), (r2, c2))
                copy = FourDominoes(self.N, state)
                copy.move(action)
                successors.append((action, copy.state))
        return successors

    def goal_test(self, state):
        """
        Checks if the given state is a goal state.
        """
        for i in range(self.N):
            for j in range(self.N):
                if i > 0 and not state[i][j].is_under(state[i - 1][j]):
                    return False
                if j > 0 and not state[i][j].is_on_the_right_of(state[i][j - 1]):
                    return False
        return True

In [19]:
def bfs_search(four_dominoes):
    start_state = four_dominoes.state
    queue = Queue()
    queue.put((start_state, []))  # (state, path)
    visited = set()
    visited.add(start_state)
    expanded_states = 0  # Track number of expanded states

    while not queue.empty():
        current_state, path = queue.get()
        expanded_states += 1  # Increment for each state expanded

        if four_dominoes.goal_test(current_state):
            return expanded_states, True

        for action, new_state in four_dominoes.successor(current_state):
            if new_state not in visited:
                visited.add(new_state)
                queue.put((new_state, path + [action]))

    return expanded_states, False  # No solution found
    
def a_star_search(four_dominoes):
    def heuristic(state):
        unmatched_edges = 0
        N = four_dominoes.N
        for i in range(N):
            for j in range(N):
                if i < N - 1 and not state[i][j].is_under(state[i + 1][j]):
                    unmatched_edges += 1
                if j < N - 1 and not state[i][j].is_on_the_right_of(state[i][j + 1]):
                    unmatched_edges += 1
        return unmatched_edges

    def get_state_id(state):
        return tuple((tile.top, tile.right, tile.bottom, tile.left) for row in state for tile in row)

    start_state = four_dominoes.state
    open_list = []
    start_state_id = get_state_id(start_state)
    heapq.heappush(open_list, (0, start_state_id, []))  # (cost, state_id, path of actions)
    visited = set()
    visited.add(start_state_id)
    expanded_states = 0  # Track number of expanded states

    while open_list:
        cost, current_state_id, path = heapq.heappop(open_list)
        expanded_states += 1  # Increment for each state expanded

        current_state = [[Domino(*current_state_id[i * four_dominoes.N + j])
                          for j in range(four_dominoes.N)]
                         for i in range(four_dominoes.N)]

        if four_dominoes.goal_test(current_state):
            return expanded_states, True

        for action, new_state in four_dominoes.successor(current_state):
            new_state_id = get_state_id(new_state)
            if new_state_id not in visited:
                visited.add(new_state_id)
                estimated_cost = cost + 1 + heuristic(new_state)
                heapq.heappush(open_list, (estimated_cost, new_state_id, path + [action]))

    return expanded_states, False  # No solution found

In [21]:
# Experiment function to gather results over multiple trials
def run_experiment_for_n(four_dominoes_class, N, algorithm, num_trials=10):
    results = {
        'expanded_states': [],
        'successes': 0
    }

    for trial in range(num_trials):
        # Initialize a random puzzle instance for the given N
        puzzle_instance = four_dominoes_class(N)
        
        # Select algorithm and track expanded states
        if algorithm == 'BFS':
            expanded_states, success = bfs_search(puzzle_instance)
        elif algorithm == 'A*':
            expanded_states, success = a_star_search(puzzle_instance)
        
        # Record metrics
        results['expanded_states'].append(expanded_states)
        if success:
            results['successes'] += 1
    
    # Calculate average expanded states
    avg_expanded_states = np.mean(results['expanded_states'])
    success_rate = (results['successes'] / num_trials) * 100  # percentage

    return avg_expanded_states, success_rate

# Running experiments for N=2, 3, 4 with both BFS and A* algorithms
experiment_results = {}

In [ ]:
for N in [2, 3, 4]:
    experiment_results[f'BFS_N={N}'] = run_experiment_for_n(FourDominoes, N, 'BFS')
    experiment_results[f'A*_N={N}'] = run_experiment_for_n(FourDominoes, N, 'A*')

print(experiment_results)

# Report template

## Solution description
The overall goal of this project was to configure the 4 sides dominoes, generate random states and test to see if the dominoes could be placed in a goal state. Methods used in this are Breadth First Search or BFS and A*. The BFS explores all of the configurations without preference. A* estimates the number of unmatched edges to prioritize promising states.

### Search space

Format of the search space consists of an NxN grid of the four sides dominoes with each of the 4 values representing the top, left, right and bottom edges, with the goal being to arrange the tiles so that the adjacent tiles would match across the grid.

Size of the grid was N size with N^2 tiles. This resulting search space would be a factorial of (N^2)!

Start state is randomly generated configurations of the tiles

Goal State as stated above would be the arrangement of the tiles so that the adjacent tiles would match across the grid.

### Successor function

How it works? Average number of successors? Etc. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Scelerisque eu ultrices vitae auctor eu augue. Tempor nec feugiat nisl pretium fusce. Sed cras ornare arcu dui vivamus arcu felis. Leo duis ut diam quam nulla porttitor massa. Lacus vestibulum sed arcu non odio euismod lacinia. Augue neque gravida in fermentum et sollicitudin ac orci. Habitant morbi tristique senectus et. Ut tellus elementum sagittis vitae et. Diam vulputate ut pharetra sit amet. Nisl nunc mi ipsum faucibus vitae aliquet nec ullamcorper sit. Nec ullamcorper sit amet risus nullam eget felis. Fringilla ut morbi tincidunt augue interdum velit euismod. Facilisis magna etiam tempor orci eu lobortis.

### Heuristic function

How it works? Admissible? Etc. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Scelerisque eu ultrices vitae auctor eu augue. Tempor nec feugiat nisl pretium fusce. Sed cras ornare arcu dui vivamus arcu felis. Leo duis ut diam quam nulla porttitor massa. Lacus vestibulum sed arcu non odio euismod lacinia. Augue neque gravida in fermentum et sollicitudin ac orci. Habitant morbi tristique senectus et. Ut tellus elementum sagittis vitae et. Diam vulputate ut pharetra sit amet. Nisl nunc mi ipsum faucibus vitae aliquet nec ullamcorper sit. Nec ullamcorper sit amet risus nullam eget felis. Fringilla ut morbi tincidunt augue interdum velit euismod. Facilisis magna etiam tempor orci eu lobortis.

## Experimental results

Results are presented in the table below. Each search was repeated 10 times for each implemented method. The maximum number of expanded states per run was set to 1000000. **Avg** shows the average number of expanded states, **Goal** shows the number of times the goal was reached in each case, **Fail #1** shows the number of times the search ended without reaching the goal, and **Fail #2** shows the number of times the search ended because the maximum number of expanded states was reached. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Scelerisque eu ultrices vitae auctor eu augue. Tempor nec feugiat nisl pretium fusce. Sed cras ornare arcu dui vivamus arcu felis. Leo duis ut diam quam nulla porttitor massa. Lacus vestibulum sed arcu non odio euismod lacinia. Augue neque gravida in fermentum et sollicitudin ac orci. Habitant morbi tristique senectus et. Ut tellus elementum sagittis vitae et. Diam vulputate ut pharetra sit amet. Nisl nunc mi ipsum faucibus vitae aliquet nec ullamcorper sit. Nec ullamcorper sit amet risus nullam eget felis. Fringilla ut morbi tincidunt augue interdum velit euismod. Facilisis magna etiam tempor orci eu lobortis.

| Puzzle size |&#124;| Uninformed | |         |         |&#124;| Informed | |         |         |
|-------------|------|-------|------|---------|---------|------|-----|------|---------|---------|
|             |&#124;| Avg   | Goal | Fail #1 | Fail #2 |&#124;| Avg | Goal | Fail #1 | Fail #2 |
| 2x2         |&#124;| 1     | 10   | 0       | 0       |&#124;| 1   | 10   | 0       | 0       |
| 3x3         |&#124;| 100   | 3    | 7       | 0       |&#124;| 10  | 7    | 3       | 0       |
| 4x4         |&#124;| 10000 | 0    | 2       | 8       |&#124;| 100 | 3    | 4       | 3       |